<a href="https://colab.research.google.com/github/Mitul2003/CLASS_ASSIGNMENT_OF_BIG_DATA/blob/main/Lab1_mat_mul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time


<p align="center"><b>NAME: MITUL DUDHAT</p>
<p align="center"><b>ID : 202318024</p>

# MATRIX MULTIPLICATION

In [ ]:



def mat_mul(A, B, b_size):
    m, n = A.shape
    p, q = B.shape

    if n != p:
        print("Matrices multiplication is not possible")
        return

    C = np.zeros((m, q))

    # Perform block matrix multiplication
    for i in range(0, m, b_size):  # row
        for j in range(0, q, b_size):  # col
            for k in range(0, n, b_size):  # inner treat
                block_A = A[i:i + b_size, k:k + b_size]  # change in row
                block_B = B[k:k + b_size, j:j + b_size]  # change in  column

                C[i:i + b_size, j:j + b_size] += np.dot(block_A, block_B)
    return C


In [ ]:
# initialize 4 rows and 4 columns in [0,1) interval

# A = np.random.rand(4, 4)
# B = np.random.rand(4, 4)

In [ ]:
# -----------main program --------------

A = np.array([
    [1, 2, 4, 1],
    [1, 0, -1, -2],
    [2, -1, 3, 1],
    [1, 2, 3, 4]
])
B = np.array([
    [-2, 2, -3, 3],
    [0, 1, -1, 2],
    [-2, -1, 0, 1],
    [4, 0, 1, 4]
])



ans = mat_mul(A, B, 1)
print(ans)

[[-6.  0. -4. 15.]
 [-8.  3. -5. -6.]
 [-6.  0. -4. 11.]
 [ 8.  1. -1. 26.]]


# TASK B

In [ ]:



# Function to make API requests for album information
def get_album_data(query,total_songs):
    # Set the API parameters
    params = {
        'q': query,
        'type': 'multi',
        'offset': '0',
        'limit': total_songs,
        'numberOfTopResults': '5',
    }

    # Set headers with your RapidAPI key
    headers = {
        'X-RapidAPI-Host': 'spotify23.p.rapidapi.com',
        'X-RapidAPI-Key': '8c80d5ae6cmshe8a9b05da100d23p19785fjsn7052c34aff62',
    }
    SPOTIFY_API_URL = 'https://spotify23.p.rapidapi.com/search/'
    # Make the API request
    response = requests.get(SPOTIFY_API_URL, headers=headers, params=params)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        search_results = response.json()
        return search_results.get('albums', {}).get('items', [])
    else:
        print(f'Error: {response.status_code}')
        return []

# Get user input for the search term
search_term = input("Enter a search term for music albums: ")
total_songs= input("Enter Total Number of Songs:")

# Make API request for album information
album_results = get_album_data(search_term,total_songs)

album_data_list = []

# Extract relevant information from the API response
for album in album_results:
    album_data_list.append({
        'Title': album['data']['name'],
        'Artists': ', '.join(artist['profile']['name'] for artist in album['data']['artists']['items']),
        'Cover Art URL': album['data']['coverArt']['sources'][0]['url'],
        'Release Year': album['data']['date']['year'],
    })

# Save the dataset to a CSV file
csv_file_path = f'{search_term}_albums_dataset.csv'
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['Title', 'Artists', 'Cover Art URL', 'Release Year']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()

    for album_data in album_data_list:
        writer.writerow(album_data)

print(f'Dataset created and saved to: {csv_file_path}')

import pandas as pd

pd.read_csv("/content/love_albums_dataset.csv")

Enter a search term for music albums: love
Enter Total Number of Songs:100
Dataset created and saved to: love_albums_dataset.csv


,Title,Artists,Cover Art URL,Release Year
0,Lover,Taylor Swift,https://i.scdn.co/image/ab67616d00001e02e787cf...,2019
1,Love,The Cult,https://i.scdn.co/image/ab67616d00001e02bafa44...,1985
2,lovely (with Khalid),"Billie Eilish, Khalid",https://i.scdn.co/image/ab67616d00001e028a3f0a...,2018
3,Love,The Beatles,https://i.scdn.co/image/ab67616d00001e0230503d...,2006
4,Love Angel Music Baby (Deluxe Version),Gwen Stefani,https://i.scdn.co/image/ab67616d00001e027c7136...,2004
...,...,...,...,...
95,Love,"Honey, Shoby",https://i.scdn.co/image/ab67616d00001e022af9a5...,2022
96,Real Love Baby,Father John Misty,https://i.scdn.co/image/ab67616d00001e02c6ad8b...,2016
97,Love,Fyze,https://i.scdn.co/image/ab67616d00001e029ebeb2...,2023
98,The Truth About Love,P!nk,https://i.scdn.co/image/ab67616d00001e029d0f0d...,2012


# TASK C

In [ ]:
def scrape_amazon(search_keyword, num_pages):
    base_url = 'https://www.amazon.in/s'
    data = []

    for page in range(1, num_pages + 1):
        params = {'k': search_keyword, 'page': page}
        webpage = requests.get(base_url, params=params)
        soup = bs(webpage.content, 'html.parser')

        products=soup.find_all('div',class_='puis-card-container s-card-container s-overflow-hidden aok-relative puis-include-content-margin puis puis-v1x9xtdrqwkf2b2sy3gv3bhvhpn s-latency-cf-section puis-card-border')
        for product in products:

            brand_name = product.find('h2').text.strip()
            rating = product.find('span', class_='a-icon-alt').text.strip() if product.find('span', class_='a-icon-alt') else 'N/A'
            reviews = product.find('span', class_='a-size-base s-underline-text').text.strip() if product.find('span', class_='a-size-base s-underline-text') else 'N/A'
            price = product.find('span', class_='a-price-whole').text.strip() if product.find('span', class_='a-price-whole') else 'N/A'
            des = product.find('h2',class_='a-size-mini a-spacing-none a-color-base s-line-clamp-2').text.strip()



            data.append({'brand_name': brand_name,'description':des , 'rating': rating, 'reviews': reviews, 'price': price})
        # time.sleep(2);

    return pd.DataFrame(data)

# Example usage
search_keyword = 'laptop'
num_pages_to_scrape = 30  # Change this to the number of pages you want to scrape
df = scrape_amazon(search_keyword, num_pages_to_scrape)
print(df)


                                           brand_name  \
0   HP Pavilion Aero AMD Ryzen 7 7735U 13.3 inch(3...   
1   ASUS VivoBook 15 (2021), 15.6-inch (39.62 cm) ...   
2   Chuwi HeroBook Pro 14.1'' Laptop, 8GB RAM 256G...   
3   Dell 15 Laptop, Intel Core i3-1115G4 Processor...   
4   Acer Aspire Lite 12th Gen Intel Core i3-1215U ...   
5   HP Victus Gaming Laptop, AMD Ryzen 7 7840HS AI...   
6   HP Chromebook X360 Intel Celeron N4120 14 inch...   
7   [Windows 11 Home] HP 255 G8 Notebook, 15.6'' H...   
8   TECNO MEGABOOK T1, Intel Core 11th Gen i3 Proc...   
9   HONOR MagicBook X16 (2024), 12th Gen Intel Cor...   
10  ASUS Vivobook 14, Intel Core i3-1115G4 11th Ge...   
11  HP Laptop 15s, Intel Celeron, 15.6-inch (39.6 ...   
12  HP 255 G9 (840T7PA) Notebook (AMD Athlon Silve...   
13  HP Laptop 15s, 12th Gen Intel Core i3-1215U, 1...   
14  HP Laptop 15s, 12th Gen Intel Core i3-1215U, 1...   
15  TECNO MEGABOOK T1, Intel Core 11th Gen i5 Proc...   
16  MSI Modern 15, AMD Ryzen 5-

In [ ]:
df.to_csv(r'Leptop.csv',index=True)

In [ ]:
df1 = pd.read_csv('Leptop.csv')
df1

,Unnamed: 0,brand_name,description,rating,reviews,price
0,0,HP Pavilion Aero AMD Ryzen 7 7735U 13.3 inch(3...,HP Pavilion Aero AMD Ryzen 7 7735U 13.3 inch(3...,4.4 out of 5 stars,18,"77,990"
1,1,"ASUS VivoBook 15 (2021), 15.6-inch (39.62 cm) ...","ASUS VivoBook 15 (2021), 15.6-inch (39.62 cm) ...",3.9 out of 5 stars,"1,276","22,990"
2,2,"Chuwi HeroBook Pro 14.1'' Laptop, 8GB RAM 256G...","Chuwi HeroBook Pro 14.1'' Laptop, 8GB RAM 256G...",3.9 out of 5 stars,"4,541","17,990"
3,3,"Dell 15 Laptop, Intel Core i3-1115G4 Processor...","Dell 15 Laptop, Intel Core i3-1115G4 Processor...",3.8 out of 5 stars,595,"37,490"
4,4,Acer Aspire Lite 12th Gen Intel Core i3-1215U ...,Acer Aspire Lite 12th Gen Intel Core i3-1215U ...,4.0 out of 5 stars,696,"34,990"
...,...,...,...,...,...,...
107,27,"ASUS Vivobook Go 14, AMD Ryzen 3 7320U, 14"" (3...","ASUS Vivobook Go 14, AMD Ryzen 3 7320U, 14"" (3...",4.0 out of 5 stars,42,"38,990"
108,28,"Dell Inspiron 3520 Laptop, Intel Core i5-1235U...","Dell Inspiron 3520 Laptop, Intel Core i5-1235U...",4.5 out of 5 stars,2,"48,450"
109,29,"ASUS Creator Series Vivobook 16X (2023), Intel...","ASUS Creator Series Vivobook 16X (2023), Intel...",4.2 out of 5 stars,13,"66,990"
110,30,"Lenovo ThinkPad E16 Intel Core i3 13th Gen 16""...","Lenovo ThinkPad E16 Intel Core i3 13th Gen 16""...",3.9 out of 5 stars,49,"49,990"
